In [1]:
import itertools

import numpy as np
import scipy as sp
import pandas as pd
import networkx as nx

import meshplot as mp
import pyvista as pv
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns

from src import shapes

# Define The Figure and the Morse Function

In [2]:
def get_linear_morse(vector=None):
    if vector is None:
        vector = np.random.random(4)
    vector = np.array(vector)
    def f(points):
        return points @ vector
    return f

direction = np.random.random(3)
direction /= np.linalg.norm(direction)

In [3]:
f = lambda p: np.linalg.norm(p, axis=-1, ord=2)
#f = lambda p: p[:, -1]
#f = lambda p: (np.random.random(3)*p).sum(axis=-1)*(np.random.random(3)*p).sum(axis=-1)

In [4]:
def cylindrical_twist(vertices, k=1.0, mode="x", scale=1.0):
    """
    Nonlinear cylindrical twist diffeomorphism on R^3.

    vertices: (n,3) array
    mode:
      - "z": angle depends on z  (theta = k * tanh(z/scale))
      - "r": angle depends on radius r (theta = k * tanh(r/scale))
    k: twist strength (radians, roughly bounded by +/-k for tanh)
    scale: controls how quickly tanh saturates
    """
    if mode == 'x':
        v = vertices[:, [1, 2, 0]]
        v = cylindrical_twist(v, k=k, mode="z", scale=scale)
        v = v[:, [2, 0, 1]]
        return v
        
        
    v = vertices.copy()
    x, y, z = v[:, 0], v[:, 1], v[:, 2]

    r = np.sqrt(x*x + y*y)

    if mode == "z":
        theta = k * np.tanh(z / scale)
    elif mode == "r":
        theta = k * np.tanh(r / scale)
    else:
        raise ValueError('mode must be "z" or "r"')

    c, s = np.cos(theta), np.sin(theta)

    v[:, 0] = c * x - s * y
    v[:, 1] = s * x + c * y
    
    return v

In [5]:
n, m = 13, 12
vertices, faces = shapes.get_halftori_bouquet(leaves=2, n=n, m=m, l0=0.9, glue=False)

vertices = cylindrical_twist(vertices, k=-0.3, scale=1.5, mode='x')

vertices, faces = shapes.split_large_edges(vertices, faces, max_length=1.0)


print(f'faces.shape = {faces.shape}')

face_mean_values = f(vertices[faces]).mean(axis=1)

p = mp.plot(vertices, faces, face_mean_values, shading={"wireframe": True})

faces.shape = (788, 3)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.0,…

In [6]:
faces_pv = np.hstack([np.full((faces.shape[0], 1), 3, dtype=faces.dtype), faces]).ravel()

mesh = pv.PolyData(vertices, faces_pv)
mesh.point_data["values"] = f(vertices)  # per-vertex scalars

p = pv.Plotter(window_size=(600, 600))
p.add_mesh(
    mesh,
    scalars="values",
    cmap="viridis",
    smooth_shading=False,   # helps show linear interpolation nicely
    show_edges=True,      # set True if you want to see triangle edges
)
p.add_scalar_bar(title="values")
p.show()

Widget(value='<iframe src="http://localhost:46251/index.html?ui=P_0x7384d9073290_0&reconnect=auto" class="pyvi…

# Paths

In [7]:
from src.ms import MorseSmale

In [8]:
#ms = MorseSmale(faces, f(vertices), vertices, forest_method='steepest')
ms = MorseSmale(faces, f(vertices), vertices, forest_method='spaning')

paths = list(ms.iterate_paths())

In [9]:
from src.vis import plot_paths, plot_segmentation_forests, plot_ms_comparition

In [10]:
pl = plot_paths(ms)
pl.show()

Widget(value='<iframe src="http://localhost:46251/index.html?ui=P_0x7384a2448320_1&reconnect=auto" class="pyvi…

In [11]:
pl = plot_segmentation_forests(ms, plot_complex=True,  point_size=16)
pl.show()

Widget(value='<iframe src="http://localhost:46251/index.html?ui=P_0x7384c0dae180_2&reconnect=auto" class="pyvi…

In [12]:
forest_methods = ['steepest', 'spaning']
mss = [MorseSmale(faces, f(vertices), vertices, forest_method=forest_method) for forest_method in forest_methods]
[ms.define_critical_points() for ms in mss]

pl = plot_ms_comparition(mss, forest_methods)
pl.show()

Widget(value='<iframe src="http://localhost:46251/index.html?ui=P_0x7384c0dafb60_3&reconnect=auto" class="pyvi…